In [1]:
def frame_to_time(frame, dt_video):
    return frame * dt_video

frame_to_time(1, 0.1)

0.1

In [2]:
class SiloToVideoScaler:
    """
    Scales coordinates between the silo space and the video pixels.
    
    Silo coordinates are up right oriented. Video coordinates are down right oriented.
    """
    def __init__(self,
                 video_width_px,
                 video_height_px,
                 silo_width_cm,
                 silo_vertical_boundaries,
                 horizontal_padding_px,
                 vertical_padding_px):
        silo_width_px = video_width_px - 2 * horizontal_padding_px
        silo_height_cm = silo_vertical_boundaries[1] - silo_vertical_boundaries[0]
        silo_height_px = video_height_px - 2 * vertical_padding_px
        
        self.coordinate_to_px_scaler = {
            'x': lambda x: horizontal_padding_px + silo_width_px / silo_width_cm * x,
            'y': lambda y: video_height_px - vertical_padding_px - silo_height_px / silo_height_cm * (y - silo_vertical_boundaries[0]),
        }

    def coordinate_to_px(self, axis, coordinate):
        return self.coordinate_to_px_scaler[axis](coordinate)


In [3]:
import cv2
import os
import re

def add_gap_walls(video_path):
    basepath = os.path.basename(video_path)
    dirname = os.path.dirname(video_path)
    pattern = r"fq(\d+)"
    match = re.search(pattern, basepath)
    if match:
        freq_number = int(match.group(1))
    else:
        raise RuntimeError()
        
    pattern = r"gap(\d+\.\d+|\d+)"
    match = re.search(pattern, basepath)
    if match:
        gap = float(match.group(1))
    else:
        raise RuntimeError()

    scaler = SiloToVideoScaler(500, 1200, 0.2, (-0.7 / 10, 0.7), 115, 81)
    cap = cv2.VideoCapture(video_path)


    # Check if video file successfully opened
    if not cap.isOpened():
        print("Error opening video file")
        exit()

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define line coordinates
    line_color = (0, 0, 0)  # Green color in BGR format
    line_thickness = 2

    # Create VideoWriter object to save output video
    output_path = os.path.join(dirname, basepath.replace('.mp4', '_walled.mp4'))  # Replace with your desired output file path
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec for output video file (may vary based on system)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    frame_number = 0
    while True:
        # Read a frame from the video file
        ret, frame = cap.read()

        # If the frame was not read successfully, break the loop
        if not ret:
            break

        start_point_x = scaler.coordinate_to_px('x', 0)
        end_point_x = scaler.coordinate_to_px('x', 0.2 / 2 - gap / 2)

        start_point_y = scaler.coordinate_to_px('y', 0) + line_thickness
        start_point = (int(start_point_x), int(start_point_y))
        end_point = (int(end_point_x), int(start_point_y))

        # Draw the line on the frame
        cv2.line(frame, start_point, end_point, line_color, line_thickness)

        start_point_x = scaler.coordinate_to_px('x', 0.2 / 2 + gap / 2)
        end_point_x = scaler.coordinate_to_px('x', 0.2)
        start_point = (int(start_point_x), int(start_point_y))
        end_point = (int(end_point_x), int(start_point_y))
        cv2.line(frame, start_point, end_point, line_color, line_thickness)

        # Display the frame
        cv2.imshow("Video with Line", frame)

        # Write the frame to the output video file
        out.write(frame)

        # Check for 'q' key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_number += 1

    # Release the video file and output video objects
    cap.release()
    out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()
    

add_gap_walls("../tp5/out/silo/silo_fq5_gap0.03_i0.mp4")